# Product pulse

##Problem statement
Infer data from support tickets to make data-informed decisions, enhance product quality, optimize customer support processes, and ultimately drive positive business outcomes, including improved customer satisfaction, increased customer retention, and business growth

##Solution
Infer qualitative and factual information such as user emotion while writing the ticket, summary, feature request, issue type, priotiy, product mentioned from support tickets using OpenAI APIs and create easy to understand visualisation basis the inferred data.


##Callouts
1. Chain of messages between the user and the agent in a support ticket can't be read at the moment as it requires some more development
2. We are using only the generic model from openAI for all tasks while there are specifc models that perform better at certain jobs[such as text-curie-003 do better and cheaper summarisation]. This will not only improve the accuracy of answers but will also result in reduced cost.
3. Frontend dashboards are yet to be done for visualing the data


##Project Summary in PR format
**Product Pulse**: *Unleash the Power of Support Data*

Product Pulse, a revolutionary startup, is proud to unveil its flagship product, Product Pulse, a game-changing solution designed to transform customer service for businesses. Leveraging advanced natural language processing and OpenAI APIs, Product Pulse extracts invaluable insights from support tickets, empowering businesses to optimize customer experiences and drive positive outcomes.

🔍 Product Pulse offers a suite of powerful features, including ticket emotion analysis, summary generation, feature request identification, issue categorization, and priority and team assignment. These features enable businesses to streamline their support processes, make data-informed decisions, and enhance overall customer satisfaction.

Visualizations based on the extracted data provide businesses with intuitive insights, including product type issue distribution, emotion distribution, issue type distribution, feature request distribution, and team-level issue distribution. These visualizations facilitate targeted improvements, effective issue resolution, and optimized resource allocation.

"Product Pulse revolutionizes the way businesses handle support tickets," said CEO of Product Pulse. "Our solution unlocks valuable insights and empowers businesses to prioritize customer satisfaction and drive growth."

Businesses can seamlessly integrate Product Pulse with their existing help desk software or securely upload their ticket data. Whether using popular help desk software like Zendesk, Freshdesk, Help Scout, or custom-built solutions, Product Pulse adapts to diverse business needs.

"Product Pulse has transformed the way we understand our customers and prioritize improvements," shared a satisfied merchant. "The invaluable insights and visualizations have enhanced our support processes and customer satisfaction."

Product Pulse, a dynamic startup, aims to revolutionize customer service and empower businesses with actionable insights. With a commitment to excellence and innovation, Product Pulse is poised to make a significant impact on businesses across industries.

##Dependencies

In [ ]:
!pip install openai
!pip install huggingface-hub
!pip install --upgrade langchain
!pip install jq

In [ ]:
import os
import json
import itertools
import pandas as pd

import langchain
from langchain.vectorstores import FAISS, Chroma
from langchain.document_loaders import JSONLoader

import openai
import uuid
import random
import time
from copy import deepcopy

In [ ]:
openai.api_key = "" #OpenAI key from your account

## DataSets

### JPMORGAN DATASET: Downloaded from kaggle

In [ ]:
path = "/content/complaints-2021-05-14_08_16_.json"
# os.path.abspath(path)
with open(os.path.abspath(path), encoding="utf-8") as f:
  data = json.load(f)

In [ ]:
tickets_df.head(2)

,_index,_type,_id,_score,_source.tags,_source.zip_code,_source.complaint_id,_source.issue,_source.date_received,_source.state,...,_source.company_response,_source.company,_source.submitted_via,_source.date_sent_to_company,_source.company_public_response,_source.sub_product,_source.timely,_source.complaint_what_happened,_source.sub_issue,_source.consumer_consent_provided
0,complaint-public-v2,complaint,3211475,0.0,None,90301,3211475,Attempts to collect debt not owed,2019-04-13T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-13T12:00:00-05:00,None,Credit card debt,Yes,,Debt is not yours,Consent not provided
1,complaint-public-v2,complaint,3229299,0.0,Servicemember,319XX,3229299,Written notification about debt,2019-05-01T12:00:00-05:00,GA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-05-01T12:00:00-05:00,None,Credit card debt,Yes,Good morning my name is XXXX XXXX and I apprec...,Didn't receive enough information to verify debt,Consent provided


In [ ]:
tickets_df["_source.complaint_id"].nunique()

78313

In [ ]:
tickets_df["_source.sub_issue"].value_counts()

Credit card company isn't resolving a dispute about a purchase on your statement    2904
Deposits and withdrawals                                                            2759
Company closed your account                                                         2020
Problem using a debit or ATM card                                                   1613
Card was charged for something you did not purchase with the card                   1056
                                                                                    ... 
Problem with additional add-on products or services purchased with the loan            1
Receiving unwanted marketing/advertising                                               1
Report shared with employer w/o consent                                                1
Problem cancelling or closing account                                                  1
Account terms and changes                                                              1
Name: _source.sub_iss

In [ ]:
data[1]["_source"]

{'tags': 'Servicemember',
 'zip_code': '319XX',
 'complaint_id': '3229299',
 'issue': 'Written notification about debt',
 'date_received': '2019-05-01T12:00:00-05:00',
 'state': 'GA',
 'consumer_disputed': 'N/A',
 'product': 'Debt collection',
 'company_response': 'Closed with explanation',
 'company': 'JPMORGAN CHASE & CO.',
 'submitted_via': 'Web',
 'date_sent_to_company': '2019-05-01T12:00:00-05:00',
 'company_public_response': None,
 'sub_product': 'Credit card debt',
 'timely': 'Yes',
 'complaint_what_happened': 'Good morning my name is XXXX XXXX and I appreciate it if you could help me put a stop to Chase Bank cardmember services. \nIn 2018 I wrote to Chase asking for debt verification and what they sent me a statement which is not acceptable. I am asking the bank to validate the debt. Instead I been receiving mail every month from them attempting to collect a debt. \nI have a right to know this information as a consumer. \n\nChase account # XXXX XXXX XXXX XXXX Thanks in advance 

In [ ]:
tickets_df["_source.complaint_what_happened"].tolist()[:40]

['',
 'Good morning my name is XXXX XXXX and I appreciate it if you could help me put a stop to Chase Bank cardmember services. \nIn 2018 I wrote to Chase asking for debt verification and what they sent me a statement which is not acceptable. I am asking the bank to validate the debt. Instead I been receiving mail every month from them attempting to collect a debt. \nI have a right to know this information as a consumer. \n\nChase account # XXXX XXXX XXXX XXXX Thanks in advance for your help.',
 'I upgraded my XXXX XXXX card in XX/XX/2018 and was told by the agent who did the upgrade my anniversary date would not change. It turned the agent was giving me the wrong information in order to upgrade the account. XXXX   changed my anniversary date from XX/XX/XXXX to XX/XX/XXXX without my consent! XXXX has the recording of the agent who was misled me.',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Chase Card was reported on XX/XX/2019. However, fraudulent application have been submitted my identity 

In [ ]:
tickets["_source"].iloc[0]

{'tags': None,
 'zip_code': '90301',
 'complaint_id': '3211475',
 'issue': 'Attempts to collect debt not owed',
 'date_received': '2019-04-13T12:00:00-05:00',
 'state': 'CA',
 'consumer_disputed': 'N/A',
 'product': 'Debt collection',
 'company_response': 'Closed with explanation',
 'company': 'JPMORGAN CHASE & CO.',
 'submitted_via': 'Web',
 'date_sent_to_company': '2019-04-13T12:00:00-05:00',
 'company_public_response': None,
 'sub_product': 'Credit card debt',
 'timely': 'Yes',
 'complaint_what_happened': '',
 'sub_issue': 'Debt is not yours',
 'consumer_consent_provided': 'Consent not provided'}

Importing openAI

### E-commerce company tickets: Generating mock tickets using GPT-turbo-3.5


#### Prompting for ticket data creation

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0.9,
        top_k=4
    )
    return response.choices[0].message["content"]

In [ ]:
prompt = f"""

The company, refund policy , support policy and return policy details for a company named WoltWatch  are mentioned in detail below with the right heading and delimers(each detail starts with "[" and ends with "]"). 

Company details
[
Company Name: "WoltWatch",
Business: "WoltWatch is a company that specializes in selling affordable smartwatches to Indian users via its website "woltwatch.in". The company offers a range of smartwatches priced between Rs. 2000 and Rs. 3000. These smartwatches are designed to provide users with various features and functionalities, including fitness tracking, notification alerts, heart rate monitoring, and more.",
User Journey: "When a user decides to purchase a smartwatch from WoltWatch, they can browse through the available models on the company's website or mobile app. Once they have selected a smartwatch, they can proceed to the checkout process, where they provide their shipping address and make the payment. After the user completes the purchase, WoltWatch works in collaboration with its delivery partners to ensure the prompt and secure delivery of the smartwatch to the user's provided address. The delivery partners employed by WoltWatch are responsible for handling the logistics and transportation of the product, ensuring that it reaches the user's location in a timely manner."
]

Refund policy for the company
[
Eligibility:

WoltWatch offers a refund on smartwatches purchased directly from their website or mobile app.
To be eligible for a refund, the customer must initiate the return within 30 days from the date of purchase.
Return Process:

Customers can request a refund by contacting WoltWatch's customer support team via email or phone.
The customer will need to provide the order details and reason for the return.
Once the return is approved, the customer will receive instructions on how to return the smartwatch.
Refund Process:

Upon receiving the returned smartwatch, WoltWatch will inspect it to ensure it is in a resalable condition.
If the smartwatch meets the return criteria, a refund will be processed within a specified period, typically within 7-10 business days.
The refund will be issued through the original payment method used during the purchase.
]

Support policy for the company
[
Contact Channels:

Customers can reach out to WoltWatch's customer support team via email, phone, or through the online contact form available on their website.
Support hours are specified on the website, and responses are typically provided within 24-48 hours.
Support Assistance:

WoltWatch's support team is available to assist customers with inquiries, technical support, and troubleshooting related to their smartwatches.
They can provide guidance on product features, usage, and compatibility.
Warranty Coverage:

WoltWatch provides a warranty for their smartwatches against manufacturing defects and malfunctions.
The warranty period and coverage details are specified on the product packaging or documentation.
Customers experiencing issues covered by the warranty can contact WoltWatch's support for assistance and potential warranty claims.
]

Return Policy for WoltWatch:
[
Eligibility:

WoltWatch accepts returns for smartwatches purchased directly from their website or mobile app.
Customers must initiate the return process within 30 days from the date of purchase.
Return Process:

Customers can request a return by contacting WoltWatch's customer support team via email or phone.
The customer will need to provide the order details and reason for the return.
Once the return is approved, the customer will receive instructions on how to return the smartwatch.
Return Conditions:

The smartwatch must be returned in its original packaging, unused, and in resalable condition.
Any accessories, manuals, or included items must also be returned.
The customer is responsible for the return shipping costs unless the return is due to a product defect or an error on WoltWatch's part.
]

For that company generate a sample support ticket with the following details:
Ticket_id (Randomised id), 
Status (Status of resolution of the ticket), created_at (exact time at which the ticket was created at), closed_at (exact time at which the ticket was resolved_at, current_owner (who is the owner of the ticket), priority(What is the expected resolution priority of the ticket),  user_closure_rating(the rating provided by the user at the end of the resolution process),  owner_team(the team which is responsible for resolving the ticket), ticket_text (the detailed explanation of the problem the user is facing.)
The ticket can be about any issue (support/delivery/product/quality/). The output should be in JSON format.
"""

response = get_completion(prompt)
print(response)

#### Iterating to create sample tickets

In [ ]:
responses = []
for i in range(1, 11):
    prompt = f"""

    The company, refund policy , support policy and return policy details for a company named WoltWatch  are mentioned in detail below with the right heading and delimers(each detail starts with "[" and ends with "]"). 

    Company details
    [
    Company Name: "WoltWatch",
    Business: "WoltWatch is a company that specializes in selling affordable smartwatches to Indian users via its website "woltwatch.in". The company offers a range of smartwatches priced between Rs. 2000 and Rs. 3000. These smartwatches are designed to provide users with various features and functionalities, including fitness tracking, notification alerts, heart rate monitoring, and more.",
    User Journey: "When a user decides to purchase a smartwatch from WoltWatch, they can browse through the available models on the company's website or mobile app. Once they have selected a smartwatch, they can proceed to the checkout process, where they provide their shipping address and make the payment. After the user completes the purchase, WoltWatch works in collaboration with its delivery partners to ensure the prompt and secure delivery of the smartwatch to the user's provided address. The delivery partners employed by WoltWatch are responsible for handling the logistics and transportation of the product, ensuring that it reaches the user's location in a timely manner."
    ]

    Refund policy for the company
    [
    Eligibility:

    WoltWatch offers a refund on smartwatches purchased directly from their website or mobile app.
    To be eligible for a refund, the customer must initiate the return within 30 days from the date of purchase.
    Return Process:

    Customers can request a refund by contacting WoltWatch's customer support team via email or phone.
    The customer will need to provide the order details and reason for the return.
    Once the return is approved, the customer will receive instructions on how to return the smartwatch.
    Refund Process:

    Upon receiving the returned smartwatch, WoltWatch will inspect it to ensure it is in a resalable condition.
    If the smartwatch meets the return criteria, a refund will be processed within a specified period, typically within 7-10 business days.
    The refund will be issued through the original payment method used during the purchase.
    ]

    Support policy for the company
    [
    Contact Channels:

    Customers can reach out to WoltWatch's customer support team via email, phone, or through the online contact form available on their website.
    Support hours are specified on the website, and responses are typically provided within 24-48 hours.
    Support Assistance:

    WoltWatch's support team is available to assist customers with inquiries, technical support, and troubleshooting related to their smartwatches.
    They can provide guidance on product features, usage, and compatibility.
    Warranty Coverage:

    WoltWatch provides a warranty for their smartwatches against manufacturing defects and malfunctions.
    The warranty period and coverage details are specified on the product packaging or documentation.
    Customers experiencing issues covered by the warranty can contact WoltWatch's support for assistance and potential warranty claims.
    ]

    Return Policy for WoltWatch:
    [
    Eligibility:

    WoltWatch accepts returns for smartwatches purchased directly from their website or mobile app.
    Customers must initiate the return process within 30 days from the date of purchase.
    Return Process:

    Customers can request a return by contacting WoltWatch's customer support team via email or phone.
    The customer will need to provide the order details and reason for the return.
    Once the return is approved, the customer will receive instructions on how to return the smartwatch.
    Return Conditions:

    The smartwatch must be returned in its original packaging, unused, and in resalable condition.
    Any accessories, manuals, or included items must also be returned.
    The customer is responsible for the return shipping costs unless the return is due to a product defect or an error on WoltWatch's part.
    ]

    For that company generate a sample support ticket with the following details:
    Ticket_id (Randomised id), 
    Status (Status of resolution of the ticket), created_at (exact time at which the ticket was created at), closed_at (exact time at which the ticket was resolved_at, current_owner (who is the owner of the ticket), priority(What is the expected resolution priority of the ticket),  user_closure_rating(the rating provided by the user at the end of the resolution process),  owner_team(the team which is responsible for resolving the ticket), ticket_text (the detailed explanation of the problem the user is facing.)
    The ticket can be about any issue (support/delivery/product/quality/warranty/refund). 
    The following rules also need to be followed
    1. The current_owner field can be from a set of 10 random names. 
    2. The ticket_text which are user's messages have to be in a text form. They can be of varied tones and it can range from happy to puzzled to disappointed to sad to angry. Also, the issues can be be of multiple types such as product issue, delivery issue, refund issue, transaction issue or any kind of issue assosciated with an e-commerce company.
    3. The ticket_text should also give out the message in a tone that reflects the user's mental state during the message sending process. The tone should also be random as different people give feedback in a different tone. The tone should also reflect the seriousness of the issue.
    4. The messages in ticket_text can be complimentary as well. The ticket_text can have feature requests made by customers to the company as well. The feedback can be about multiple products as well. The feedback can be as an escalation message as well.
    5. Prioirity can be one among (High, Medium, Low)
    6. The output should be in JSON format. 
    7. Status can be one among (Open, Assigned, In-process, Blocked, Resolved)
    8. For a closed ticket the user can give a rating from 1 to 5 where 1 means un-satisfactory and 5 means super-satisfied
    9. Closed_at can have a timestamp if status is resolved else it should be null. 
    1o. created_at and closed_at timestamp should be in a standardard timestamp format
    10. The ticket_id should be in the format TY<random_6_digit_number>
    11.The owner_team can be any team randing from support to product to supply chain. This should be inferred from the ticket_text.
    The output should be in JSON format.
    """
    response = get_completion(prompt)
    print(i, response, "\n")
    responses.append(response)

In [ ]:
responses = [[
{
    "Ticket_id": "TY123456",
    "Status": "Open",
    "Created_at": "2022-07-12 09:00:00",
    "Closed_at": None,
    "Current_owner": "John Doe",
    "Priority": "High",
    "User_closure_rating": None,
    "Owner_team": "Product",
    "Ticket_text": "Hi there, I recently purchased a WoltWatch from your website and I'm having some trouble setting it up. The instructions are a bit confusing. Can you please help me out? Thanks!"
},
{
    "Ticket_id": "TY234567",
    "Status": "In-process",
    "Created_at": "2022-07-13 11:30:00",
    "Closed_at": None,
    "Current_owner": "Sarah Lee",
    "Priority": "Medium",
    "User_closure_rating": None,
    "Owner_team": "Support",
    "Ticket_text": "Hello, I ordered a smartwatch from WoltWatch a few days ago and I still haven't received it. Can you please check on the status of my order? Thank you."
},
{
    "Ticket_id": "TY345678",
    "Status": "Assigned",
    "Created_at": "2022-07-14 14:15:00",
    "Closed_at": None,
    "Current_owner": "Mike Chen",
    "Priority": "High",
    "User_closure_rating": None,
    "Owner_team": "Delivery",
    "Ticket_text": "Hi, I received my smartwatch from WoltWatch but it was damaged during shipping. Can you please help me initiate a return or replace the product? Thank you."
},
{
    "Ticket_id": "TY456789",
    "Status": "Blocked",
    "Created_at": "2022-07-15 16:45:00",
    "Closed_at": None,
    "Current_owner": "Emma Wilson",
    "Priority": "Low",
    "User_closure_rating": None,
    "Owner_team": "Finance",
    "Ticket_text": "Hello, I recently purchased a smartwatch from WoltWatch and there seems to be an issue with my payment. The amount has been debited from my account but the order status shows as unpaid. Can you please help me resolve this issue? Thank you."
},
{
    "Ticket_id": "TY567890",
    "Status": "Resolved",
    "Created_at": "2022-07-16 10:30:00",
    "Closed_at": "2022-07-18 15:00:00",
    "Current_owner": "Julia Davis",
    "Priority": "Medium",
    "User_closure_rating": 4,
    "Owner_team": "Support",
    "Ticket_text": "Hi, I've been using my WoltWatch for a few weeks now and I have some feedback. It would be great if the watch could track sleep patterns as well. Also, the battery life seems to be shorter than what was advertised. Can you please pass this on to the product team? Thanks!"
}
] ,
 {
    "ticket_id": "TY123456",
    "status": "Open",
    "created_at": "2021-09-01T10:00:00Z",
    "closed_at": None,
    "current_owner": "John",
    "priority": "High",
    "user_closure_rating": None,
    "owner_team": "Support",
    "ticket_text": "Hi there, I received my smartwatch today and it seems to have an issue with the heart rate monitor. The readings seem inaccurate and I'm not sure if it's a defect or if I'm doing something wrong. Could you please help me with this? Thank you."
} ,
{
    "ticket_id": "TY123456",
    "status": "Open",
    "created_at": "2022-03-03 09:30:00",
    "closed_at": None,
    "current_owner": "John",
    "priority": "High",
    "user_closure_rating": None,
    "owner_team": "Support",
    "ticket_text": "Hi, I recently bought a smartwatch from your website and I am facing some issues. The heart rate monitor seems to be inaccurate and I am not receiving any notifications on my phone. Please help me resolve this issue as soon as possible. Thank you."
} ,
{
    "ticket_id": "TY123456",
    "status": "Assigned",
    "created_at": "2022-01-01 10:00:00",
    "closed_at": None,
    "current_owner": "John Doe",
    "priority": "High",
    "user_closure_rating": None,
    "owner_team": "Support",
    "ticket_text": "Hi there, I recently purchased a WoltWatch from your website and it seems to be malfunctioning. The heart rate monitor doesn't seem to be working properly and the screen is flickering. Can you please help me with this issue as soon as possible? It's affecting my daily workouts and I really need a functional smartwatch. Thank you."
} ,
{
    "Ticket_id": "TY123456",
    "Status": "Open",
    "created_at": "2022-01-01 10:00:00",
    "closed_at": None,
    "current_owner": "John Doe",
    "priority": "High",
    "user_closure_rating": None,
    "owner_team": "Support",
    "ticket_text": "Hi, I recently purchased a smartwatch from WoltWatch and I'm having trouble with the heart rate monitor. It doesn't seem to be tracking my heart rate accurately. Can you please help me resolve this issue as soon as possible? Thank you."
} ,
{
  "Ticket_id": "TY123456",
  "Status": "Open",
  "Created_at": "2022-04-01 10:00:00",
  "Closed_at": None,
  "Current_owner": "John Doe",
  "Priority": "High",
  "User_closure_rating": None,
  "Owner_team": "Support",
  "Ticket_text": "Hi there, I recently purchased a smartwatch from WoltWatch and I am facing issues with the battery life. The watch hardly lasts for a day even on low usage. This is really disappointing as I had high hopes for the product. Can you please help me resolve this issue as soon as possible? Thank you."
} ,
{
    "Ticket_id": "TY123456",
    "Status": "Open",
    "created_at": "2022-01-01T12:00:00Z",
    "closed_at": None,
    "current_owner": "John Smith",
    "priority": "High",
    "user_closure_rating": None,
    "owner_team": "Support",
    "ticket_text": "Hi, I recently purchased a smartwatch from WoltWatch and it seems to be having issues with tracking my steps accurately. I've tried resetting the device and double-checked the settings, but the issue persists. Can you please help me resolve this as soon as possible? Thanks!"
} ,
{
  "Ticket_id": "TY123456",
  "Status": "Open",
  "created_at": "2021-08-10 14:30:00",
  "closed_at": None,
  "current_owner": "Jane Smith",
  "priority": "High",
  "user_closure_rating": None,
  "owner_team": "Support",
  "ticket_text": [
    {
      "message": "Hey there! I just received my WoltWatch and it seems to be defective. The screen won't turn on.",
      "timestamp": "2021-08-10 14:30:00",
      "tonality": "Puzzled"
    },
    {
      "message": "I'm really disappointed with the quality of the product. I was looking forward to using it, but it won't even turn on.",
      "timestamp": "2021-08-10 14:35:00",
      "tonality": "Disappointed"
    },
    {
      "message": "Is there any way to get a refund or exchange for a new one?",
      "timestamp": "2021-08-10 15:00:00",
      "tonality": "Concerned"
    }
  ]
} ,
{
  "Ticket_id": "TY123456",
  "Status": "Open",
  "created_at": "2021-10-10 10:00:00",
  "closed_at": None,
  "current_owner": "John Doe",
  "priority": "High",
  "user_closure_rating": None,
  "owner_team": "Support",
  "ticket_text": "Hello WoltWatch team, I recently purchased one of your smartwatches and I'm having trouble syncing it with my phone. I've tried following the instructions in the manual but it doesn't seem to work. Can you please help me resolve this issue? Thank you."
},
{
 "Ticket_id": "TY123456",
 "Status": "Open",
 "Created_at": "2022-12-31 10:30:00",
 "Closed_at": None,
 "Current_owner": "John",
 "Priority": "High",
 "User_closure_rating": None,
 "Owner_team": "Support",
 "Ticket_text": "Hi team, I received my WoltWatch yesterday and noticed that it's not syncing with my phone. I followed the instructions in the manual but it's still not working. Please help me resolve this issue as soon as possible. Thanks!"
} 
]

####
 Converting generated tickets into a data frame

In [ ]:
# convert responses to a dataframe
responses = list(itertools.chain.from_iterable([[response] if isinstance(response, dict) else response for response in responses]))

responses = [{k.lower(): v for k, v in response.items()} for response in responses]

for response in responses:
  if isinstance(response["ticket_text"], list):
    response["ticket_text"] = " ".join([i["message"] for i in response["ticket_text"]])

df = pd.DataFrame.from_dict(responses)
df.head(2)

,ticket_id,status,created_at,closed_at,current_owner,priority,user_closure_rating,owner_team,ticket_text
0,TY123456,Open,2022-07-12 09:00:00,None,John Doe,High,NaN,Product,"Hi there, I recently purchased a WoltWatch fro..."
1,TY234567,In-process,2022-07-13 11:30:00,None,Sarah Lee,Medium,NaN,Support,"Hello, I ordered a smartwatch from WoltWatch a..."


In [ ]:
df.shape

(14, 9)

In [ ]:
df

,ticket_id,status,created_at,closed_at,current_owner,priority,user_closure_rating,owner_team,ticket_text
0,TY123456,Open,2022-07-12 09:00:00,None,John Doe,High,NaN,Product,"Hi there, I recently purchased a WoltWatch fro..."
1,TY234567,In-process,2022-07-13 11:30:00,None,Sarah Lee,Medium,NaN,Support,"Hello, I ordered a smartwatch from WoltWatch a..."
2,TY345678,Assigned,2022-07-14 14:15:00,None,Mike Chen,High,NaN,Delivery,"Hi, I received my smartwatch from WoltWatch bu..."
3,TY456789,Blocked,2022-07-15 16:45:00,None,Emma Wilson,Low,NaN,Finance,"Hello, I recently purchased a smartwatch from ..."
4,TY567890,Resolved,2022-07-16 10:30:00,2022-07-18 15:00:00,Julia Davis,Medium,4.0,Support,"Hi, I've been using my WoltWatch for a few wee..."
5,TY123456,Open,2021-09-01T10:00:00Z,None,John,High,NaN,Support,"Hi there, I received my smartwatch today and i..."
6,TY123456,Open,2022-03-03 09:30:00,None,John,High,NaN,Support,"Hi, I recently bought a smartwatch from your w..."
7,TY123456,Assigned,2022-01-01 10:00:00,None,John Doe,High,NaN,Support,"Hi there, I recently purchased a WoltWatch fro..."
8,TY123456,Open,2022-01-01 10:00:00,None,John Doe,High,NaN,Support,"Hi, I recently purchased a smartwatch from Wol..."
9,TY123456,Open,2022-04-01 10:00:00,None,John Doe,High,NaN,Support,"Hi there, I recently purchased a smartwatch fr..."


In [ ]:
df["ticket_text"].tolist()[:2]

["Hi there, I recently purchased a WoltWatch from your website and I'm having some trouble setting it up. The instructions are a bit confusing. Can you please help me out? Thanks!",
 "Hello, I ordered a smartwatch from WoltWatch a few days ago and I still haven't received it. Can you please check on the status of my order? Thank you."]

## Inferring from the ticket data set using GPT-3.5-turbo

Basis ticket text data we are inferring the following details at a ticket level
1. Emotion/sentiment of the user from the ticket message
2. Summary of the ticket message in 5-10 words
3. Feature requests from the ticket
4. Type of issue (Pre-sales Inquiries, Technical Issues, Product Defects, Billing and Payment, Shipping and Delivery, Returns and Exchanges, Warranty and Repair, Account Management, Feedback and Suggestions, Misc Inquiries)
5. Priority of the ticket (Low, Medium, High)
6. The right team to solve this message (Low, Medium, High)
7. The product with the issue mentioned in the ticket

In [ ]:
# function to get inference from llm for the prompt given
def get_inference(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

In [ ]:
prompts = {
"emotion_prompt": """
You are an analyst in WoltWatch. You are analysing the support tickets. Determine the emotion of the message that is in the \
text delimited by triple backticks. The response should have only one word.
Text: 
""",
"summarization_prompt": """
You are an analyst in WoltWatch. You are analysing the support tickets. Summarise the message that is in the \
text delimited by triple backticks. The word limit is 5 words.
Text: 
""",
"feature_request_prompt": """
You are an analyst in WoltWatch. You are analysing the support tickets. Determine any explicit feature requested by user in the \
text delimited by triple backticks. If there is no feature, communnicate that 'No feature requested'. The word limit is 5 words
Text: 
""",
"issue_type_prompt": """
You are an analyst in WoltWatch. You are analysing the support tickets. Determine the type of issue in the \
text delimited by triple backticks. The word limit is 3 words. The issue type should be among (Pre-sales Inquiries, Technical Issues, Product Defects, Billing and Payment, Shipping and Delivery, Returns and Exchanges, Warranty and Repair, Account Management, Feedback and Suggestions, Misc Inquiries)
Text: 
""",
"priority_inferred_prompt": """
You are an analyst in WoltWatch. You are analysing the support tickets. Determine the priority of the issue in the \
text delimited by triple backticks. The priority should be one word and should be among (Low, Medium, High) . Ensure that the tonality and the seriousness of the issue is taken into context. The answer should not have the word priority.
Text: 
""",
"team_prompt": """
You are an analyst in WoltWatch. You are analysing the support tickets. Determine the right team that the customer support team has to involve to solve the issue in the \
text delimited by triple backticks. The answer should only be 2 words and the teams should be one among (Product Development,
Software Engineering,
Operations,
Marketing,
Sales,
QA,
R&D)
Text: 
""",
"product_type_prompt": """
You are an analyst in WoltWatch. You are analysing the support tickets. Determine the model that has the issue mentioned in the \
text delimited by triple backticks. If you are not able to determing the model, give out 'NA' as the answer. The answer can only be 1 word (model) or 'NA'
Text: 
"""
}

In [ ]:
# run the ticket text through all the prompts and save them as separate fields
for field, prompt in prompts.items():
  print("Running field: {}".format(field))
  try:
    df[field] = df["ticket_text"].apply(lambda x: get_inference(prompt + " '''" + x + "'''"))
  except Exception as e:
    time.sleep(20)
    df[field] = df["ticket_text"].apply(lambda x: get_inference(prompt + " '''" + x + "'''"))

Running field: emotion_prompt
Running field: summarization_prompt
Running field: feature_request_prompt
Running field: issue_type_prompt
Running field: priority_inferred_prompt
Running field: team_prompt
Running field: product_type_prompt


In [ ]:
final_df = deepcopy(df)

In [ ]:
# remove _prompt from column name
final_df.columns = [col if "prompt" not in col else col.split("_prompt")[0] for col in final_df.columns]

# create unique ticket ids
final_df["ticket_id"] = [str(uuid.uuid4()) for i in range(len(final_df))]

In [ ]:
final_df

,ticket_id,status,created_at,closed_at,current_owner,priority,user_closure_rating,owner_team,ticket_text,emotion,summarization,feature_request,issue_type,priority_inferred,team,product_type
0,dccefee7-204c-4447-b7e6-9cb1dae161d0,Open,2022-07-12 09:00:00,None,John Doe,High,NaN,Product,"Hi there, I recently purchased a WoltWatch fro...",Confused.,Help with setting up watch.,No feature requested.,Technical Issues.,Low,Product Development,NA
1,3a605fa0-de46-488b-a942-5e3a789bc507,In-process,2022-07-13 11:30:00,None,Sarah Lee,Medium,NaN,Support,"Hello, I ordered a smartwatch from WoltWatch a...",Impatient.,"Order not received, check status.",No feature requested.,Shipping and Delivery.,Low,Operations team,NA
2,c1b9f18f-7ee9-4a8c-82ba-f5ad8bf2b8de,Assigned,2022-07-14 14:15:00,None,Mike Chen,High,NaN,Delivery,"Hi, I received my smartwatch from WoltWatch bu...",Concerned.,"Damaged smartwatch, initiate return.",No feature requested.,Shipping and Delivery.,Medium,Operations team,NA
3,4c9e4c1c-5fdd-4986-a261-994a60e6a5bf,Blocked,2022-07-15 16:45:00,None,Emma Wilson,Low,NaN,Finance,"Hello, I recently purchased a smartwatch from ...",Concerned.,Payment issue with order.,No feature requested.,Billing and Payment.,High,Operations team,NA
4,861bb1b4-aa98-4bb8-85d5-b2bdec9d97cc,Resolved,2022-07-16 10:30:00,2022-07-18 15:00:00,Julia Davis,Medium,4.0,Support,"Hi, I've been using my WoltWatch for a few wee...",Suggestion.,Feedback on WoltWatch features.,Sleep tracking feature requested.,Feedback and Suggestions.,Medium,Product Development,NA
5,0871ab43-66a8-4552-8512-31edf0207a59,Open,2021-09-01T10:00:00Z,None,John,High,NaN,Support,"Hi there, I received my smartwatch today and i...",Concerned.,Inaccurate heart rate monitor readings.,No feature requested.,Product Defects.,Medium,Product Development,smartwatch
6,5876b9bb-10e8-47e3-9c65-d659248a3bb5,Open,2022-03-03 09:30:00,None,John,High,NaN,Support,"Hi, I recently bought a smartwatch from your w...",frustrated,Smartwatch issues reported. Need assistance.,"Heart rate monitor, notifications.",Technical Issues.,Medium,Product Development,smartwatch
7,62596f54-17bf-48d6-a372-41ac35c0275c,Assigned,2022-01-01 10:00:00,None,John Doe,High,NaN,Support,"Hi there, I recently purchased a WoltWatch fro...",frustration,"Malfunctioning WoltWatch, heart rate monitor.",Heart rate monitor malfunctioning.,Product Defects.,High,Product Development,WoltWatch
8,28f23c53-abf7-47a8-b5c8-a791d8b3fb82,Open,2022-01-01 10:00:00,None,John Doe,High,NaN,Support,"Hi, I recently purchased a smartwatch from Wol...",frustration,Heart rate monitor accuracy issue.,Heart rate monitor accuracy issue.,Technical Issues.,Medium,Product Development,smartwatch
9,8e079e32-ec54-4eb2-af4a-02ea2a36214b,Open,2022-04-01 10:00:00,None,John Doe,High,NaN,Support,"Hi there, I recently purchased a smartwatch fr...",Disappointed.,"Battery life issue, needs help.",No feature requested.,Product Defects.,High,Product Development,NA


## Creating visualisations based on the inferred data

We will be creating the following visualisations basis the inferred data:
1. Issue distribution at a product type level
2. Emotion distribution
3. Issue distribution at a issue type level
4. Feature request distribution at a issue type level
5. Issue distribution at a team level

In [ ]:
data = final_df

In [ ]:
# Group the data by 'product_type' and count the number of tickets for each product
tickets_by_product = data.groupby('product_type')['ticket_id'].count().reset_index()

# Rename the columns for the desired output format
tickets_by_product.columns = ['Model', 'Number of tickets']

# Sort the data by the number of tickets in descending order
tickets_by_product = tickets_by_product.sort_values(by='Number of tickets', ascending=False)
print(tickets_by_product)

# Display the product with the most tickets
top_product = tickets_by_product.head(1)

# Print the resulting table
print(top_product)

        Model  Number of tickets
0          NA                  8
3  smartwatch                  3
1  Smartwatch                  2
2   WoltWatch                  1
  Model  Number of tickets
0    NA                  8


In [ ]:
# Group the data by 'emotion' and count the number of tickets for each emotion
tickets_by_emotion = data.groupby('emotion')['ticket_id'].count().reset_index()

# Rename the columns for the desired output format
tickets_by_emotion.columns = ['Emotion', 'Number of tickets']

# Sort the data by the number of tickets in descending order
tickets_by_emotion = tickets_by_emotion.sort_values(by='Number of tickets', ascending=False)

# Display the emotion with the most tickets
top_emotion = tickets_by_emotion.head(1)

# Print the resulting table
print(top_emotion)

# Define a function to classify emotions into good, neutral, and bad categories
def classify_emotion(emotion):
    good_emotions = ['happy', 'satisfied', 'relieved']
    neutral_emotions = ['neutral', 'indifferent', 'mixed']
    bad_emotions = ['angry', 'sad', 'frustrated', 'disappointed']

    if emotion in good_emotions:
        return 'good'
    elif emotion in neutral_emotions:
        return 'neutral'
    else:
        return 'bad'

# Apply the classification function to the 'Emotion' column and create a new column 'Emotion Category'
tickets_by_emotion['Emotion Category'] = tickets_by_emotion['Emotion'].apply(classify_emotion)

# Print the updated table with emotion categories
print(tickets_by_emotion)

       Emotion  Number of tickets
6  frustration                  4
         Emotion  Number of tickets Emotion Category
6    frustration                  4              bad
0     Concerned.                  3              bad
2  Disappointed.                  2              bad
5     frustrated                  2              bad
1      Confused.                  1              bad
3     Impatient.                  1              bad
4    Suggestion.                  1              bad


In [ ]:
# Group the data by 'issue_type' and count the number of tickets for each issue
tickets_by_issue = data.groupby('issue_type')['ticket_id'].count().reset_index()

# Rename the columns for the desired output format
tickets_by_issue.columns = ['Issue', 'Number of tickets']

# Sort the data by the number of tickets in descending order
tickets_by_issue = tickets_by_issue.sort_values(by='Number of tickets', ascending=False)

# Display the issue with the most tickets
top_issue = tickets_by_issue.head(1)

# Print the resulting table
print(top_issue)

               Issue  Number of tickets
4  Technical Issues.                  6


In [ ]:
# Group the data by 'feature_request' and count the number of tickets for each issue
feautre_request = data.groupby('feature_request')['ticket_id'].count().reset_index()

# Rename the columns for the desired output format
feautre_request.columns = ['Feature Request', 'Number of tickets']

# Sort the data by the number of tickets in descending order
feautre_request = feautre_request.sort_values(by='Number of tickets', ascending=False)
print(feautre_request)

# Display the feature request with the most tickets
top_feature_request = feautre_request.head(1)

# Print the resulting table
print(top_feature_request)

                      Feature Request  Number of tickets
3               No feature requested.                  8
0  Heart rate monitor accuracy issue.                  1
1  Heart rate monitor malfunctioning.                  1
2  Heart rate monitor, notifications.                  1
4   Sleep tracking feature requested.                  1
5    Step tracking feature requested.                  1
6          Syncing feature requested.                  1
         Feature Request  Number of tickets
3  No feature requested.                  8


In [ ]:
# Group the data by 'team' and count the number of tickets for each product
tickets_by_team = data.groupby('team')['ticket_id'].count().reset_index()

# Rename the columns for the desired output format
tickets_by_team.columns = ['Team', 'Number of tickets']

# Sort the data by the number of tickets in descending order
tickets_by_team = tickets_by_team.sort_values(by='Number of tickets', ascending=False)
print(tickets_by_team)

# Display the product with the most tickets
top_team = tickets_by_team.head(1)
# Print the resulting table
print(top_team)

                   Team  Number of tickets
1   Product Development                  9
0       Operations team                  3
2  Software Engineering                  2
                  Team  Number of tickets
1  Product Development                  9
